In [3]:
import numpy as np
import re
from IPython.display import clear_output

from keras.layers import Dense, LSTM, Input, Embedding, Dropout
from keras.utils import np_utils
from keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam, RMSprop
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.callbacks import LambdaCallback

In [24]:
t = "Hector y Tito\
Luny Tunes y Noriega\
Con más"
len(t)

40

In [35]:
filename="pruebas.txt"

In [42]:
import codecs

with codecs.open(filename, encoding='ISO-8859-1') as f:
  text = f.read()

In [43]:
seq_length = 10
start_story = '|'*seq_length

In [48]:
text=text.lower()
text=start_story+text
#text=text.replace('\n\n\n\n\n', start_story)
#text=text.replace('\n\n\n\n\n', start_story)
text=text.replace('\n', ' xoxo ')
text=re.sub('  +', '. ', text).strip()
text=text.replace('..', '.')

text=re.sub('([!"#$%&()*+,-./:;<=>¿?@[\]^_`{|}~])\'', r' \1 ', text)
text=re.sub('\s{2,}', ' ', text)


In [49]:
# TOKENIZATION
tokenizer = Tokenizer(char_level=False, filters='')
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) +1
token_list = tokenizer.texts_to_sequences([text])[0]


In [68]:
"pudin"

'pudin'

In [60]:
import numpy as np 
from keras.utils import np_utils

def generate_sequences(token_list, step):
  X = []
  y = []
  for i in range(0,len(token_list) - seq_length,step):
    X.append(token_list[i:i + seq_length])
    y.append(token_list[i + seq_length])
  #esta funcion lo que hace es convertir un vector en una matriz tipo binaria
  y = np_utils.to_categorical(y,num_classes = total_words)

  num_seq = len(X)
  print('Number of sequences:',num_seq, "\n")

  return X,y,num_seq

step = 4
seq_length = 20
X,y,num_seq = generate_sequences(token_list,step)

X = np.array(X)
y = np.array(y)


Number of sequences: 104051 



In [61]:
from keras.layers import Dense, LSTM, Input, Embedding, Dropout
from keras.models import Model
from tensorflow.keras.optimizers import RMSprop
n_units = 256
embedding_size = 100

text_in=Input(shape= (None,))
#Embedding ocupa una dimensión de entrada que es el total de palabras que tenemos
# y una de salida que en este caso es de 100. 
#Lo que hace esta función es convertir la entrada de vectores
#binarios a un vector continuo denso
#Estos vectores son una proyeccion de las palabras a un espacio continuo
x=Embedding(total_words, embedding_size)(text_in)
#El numero de unidades es el número de unidades paralelas  'identicas' pero
#que cada una va aprender algo diferente.
x = LSTM(n_units, return_sequences = True)(x)
x = LSTM(n_units)(x)
#Se pueden apilar redes LSTM para aprender mas caracteristicas
x=Dropout(0.2)(x)
text_out=Dense(total_words, activation='softmax')(x)

model=Model(text_in, text_out)

#opti=RMSprop(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer="RMSprop")

#Se usaron solo 10 epocas porque tardaba mucho en compilar con 100
epochs=150
batch_size=32
model.fit(X, y, epochs=epochs, batch_size=batch_size, shuffle=True)

model.summary()

Epoch 1/150
3252/3252 [==============================] - 791s 238ms/step - loss: 5.9054
Epoch 2/150
3252/3252 [==============================] - 767s 236ms/step - loss: 5.4317
Epoch 3/150
3252/3252 [==============================] - 776s 239ms/step - loss: 5.2719
Epoch 4/150
3252/3252 [==============================] - 772s 237ms/step - loss: 5.2219
Epoch 5/150
3252/3252 [==============================] - 769s 236ms/step - loss: 5.4666
Epoch 6/150
3252/3252 [==============================] - 771s 237ms/step - loss: 5.7888
Epoch 7/150
3252/3252 [==============================] - 775s 238ms/step - loss: 5.9377
Epoch 8/150
3252/3252 [==============================] - 780s 240ms/step - loss: 5.9764
Epoch 9/150
3252/3252 [==============================] - 775s 238ms/step - loss: 5.9679
Epoch 10/150
3252/3252 [==============================] - 782s 240ms/step - loss: 5.9325
Epoch 11/150
3252/3252 [==============================] - 769s 236ms/step - loss: 5.8318
Epoch 12/150
3252/3252 [======

3252/3252 [==============================] - 792s 243ms/step - loss: 3.9005
Epoch 94/150
3252/3252 [==============================] - 779s 240ms/step - loss: 3.8889
Epoch 95/150
3252/3252 [==============================] - 778s 239ms/step - loss: 3.8903
Epoch 96/150
3252/3252 [==============================] - 797s 245ms/step - loss: 3.8867
Epoch 97/150
3252/3252 [==============================] - 770s 237ms/step - loss: 3.8879
Epoch 98/150
3252/3252 [==============================] - 775s 238ms/step - loss: 3.8670
Epoch 99/150
3252/3252 [==============================] - 773s 238ms/step - loss: 3.8624
Epoch 100/150
3252/3252 [==============================] - 787s 242ms/step - loss: 3.8432
Epoch 101/150
3252/3252 [==============================] - 775s 238ms/step - loss: 3.8572
Epoch 102/150
3252/3252 [==============================] - 800s 246ms/step - loss: 3.8510
Epoch 103/150
3252/3252 [==============================] - 780s 240ms/step - loss: 3.8423
Epoch 104/150
3252/3252 [=====

In [62]:
#La temperatura sirve para hacer más determinista el modelo. Una temperatura alta
#hace al modelo más diverso en las respuestas, mas sensible a los ejemplos
#pero tambien mas propenso a los errores
def sample_with_temp(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    #Se retorna el argumento con la probabilidad mas alta
    return np.argmax(probas)



def generate_text(seed_text, next_words, model, max_sequence_len, temp):
    output_text = seed_text
    seed_text = start_story + seed_text

    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = token_list[-max_sequence_len:]
        #print(token_list)
        token_list = np.reshape(token_list, (1, max_sequence_len))
        
        probs = model.predict(token_list, verbose=0)[0]
        y_class = sample_with_temp(probs, temperature = temp)
        
        output_word = tokenizer.index_word[y_class] if y_class>0 else ''
        #Si se obtiene un "|" es que el modelo ya quiere terminar la historia
        if output_word == "|":
            break
        seed_text += output_word + ' '
        output_text += output_word + ' '          
            
    return output_text



In [75]:
#La semilla tiene que se run texto con 20 palabras que es la longitud de secuencia
seed_text = "Bebe eres increible en la cama y yo te quiero comer tu cuerpo es todo lo que necesito para seguir contigo en la playa"
gen_words = 500
temp = 0.1 
cancion = generate_text(seed_text, gen_words, model, seq_length, temp)


In [76]:
cancion=cancion.replace('xoxo', ' \n ')
cancion

'Bebe eres increible en la cama y yo te quiero comer tu cuerpo es todo lo que necesito para seguir contigo en la playa \n  calle no es ( oh )  \n  lo que tú digas que no piso sudor , les saben , yeah - real ( un par de un destino )  \n   \n  si se siente la noche ( luny - mientras los quieras )  \n  cuando yo me ( de ti )  \n  quiero que me more mirando  \n  les vas hasta que se pierde ando ( wuh )  \n   \n  yo tengo una ma bien que yo te todos suena ( yeah )  \n  que tú quieres de mí , yo por eso  \n  por favor  \n  ( jajajaja )  \n  qué sabes que yo no hay miedo  \n  con el que me dé sudando , que tu cuerpo se dé "  \n  no se pa\' hacer y tú y mi vida  \n  esta para que nos de la vida ?  \n  tu ? tú es una duro yo lo que yo  \n  muy bien  \n   \n  que me da ( ¡duro ! ) , por todo tú te amo y  \n  y déjame tenerte en el barrio ( tú te juro )  \n  cuando yo te ( oh - oh )  \n  yo te quiero ver ( oh - oh , oh )  \n  y lo he que estamos me me  \n   \n  sin par de que vengo  \n  pa\' que 

In [71]:
nombres = open("Cancion2.txt","w")
nombres.write(cancion)
nombres.close()


In [77]:
model.save("lsrm.h5")